# AutoEncoder2d Demo

To demonstrate ``torch_tools.AutoEncoder2d``, we use it to encode and decode MNIST images.

In [1]:
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, RandomRotation

from torch.utils.data import DataLoader

batch_size = 32


training_tfms = Compose([ToTensor(), RandomRotation(180)])
valid_tfms = Compose([ToTensor()])


train_set = MNIST(
    root="/home/jim/storage/mnist/",
    train=True,
    download=True,
    transform=training_tfms,
)
valid_set = MNIST(
    root="/home/jim/storage/mnist/",
    train=False,
    download=True,
    transform=valid_tfms,
)

print(f"There are {len(train_set)} training items.")
print(f"There are {len(valid_set)} validation items.")

train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_set, shuffle=False, batch_size=batch_size)

There are 60000 training items.
There are 10000 validation items.


In this demo, we want bother with a validation loop during training, and will just use the validation set to do some inference at the end. Let's instantiate the model.

In [3]:
from torch.cuda import is_available
from torch_tools import AutoEncoder2d

DEVICE = "cuda" if is_available() else "cpu"

model = AutoEncoder2d(in_chans=3, out_chans=3, num_layers=3).to(DEVICE)

print(model)

AutoEncoder2d(
  (encoder): Encoder2d(
    (0): DoubleConvBlock(
      (0): ConvBlock(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.1)
      )
      (1): ConvBlock(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.1)
      )
    )
    (1): DownBlock(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConvBlock(
        (0): ConvBlock(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): LeakyReLU(negative_slope=0.1)
        )
        (1): ConvBlock(
          (0): Conv2d(128

Now, we set up the optimiser and loss function.

In [ ]:
from torch.optim import Adam
from torch.nn import L1Loss

optimiser = Adam(model.parameters(), lr=1e-4)

loss_func = L1Loss()

And now we write our training loop.

In [ ]:
from torch.nn import Module


def train_one_epoch(
    model: Module,
    data_loader: DataLoader,
    optimiser: Adam,
    loss_func: L1Loss,
) -> float:
    """Train ``model`` for a single epoch.

    Parameters
    ----------
    model : Module
        AutoEncoder model.
    data_loader : DataLoader
        Training data loader.
    optimiser : Adam
        Adam optimiser.
    loss_func : L1Loss
        The L1 loss function.
    
    Parameters
    ----------
    float
        Mean loss per item.

    """
    model.train()
    running_loss = 0.0
    for batch, _ in data_loader:

        optimiser.zero_grad()

        batch = batch.to(DEVICE)

        preds = model(batch).tanh()

        loss = loss_func(preds, batch)

        loss.backward()

        optimiser.step()

        running_loss += loss.item()
    return running_loss / len(data_loader)